# Grammars


## Definitons

We use the following terms:

* The _alphabet_ is the set all of symbols in the input language. For example,
  in this post, we use all ASCII characters as alphabet.
  
* A _terminal_ is a single alphabet symbol. Note that this is slightly different
  from usual definitions (done here for ease of parsing). (Usually a terminal is
  a contiguous sequence of symbols from the alphabet. However, both kinds of
  grammars have a one to one correspondence, and can be converted easily.)
  
  For example, `x` is a terminal symbol.

* A _nonterminal_ is a symbol outside the alphabet whose expansion is _defined_
  in the grammar using _rules_ for expansion.

  For example, `<term>` is a nonterminal in the below grammar.
  
* A _term_ is a terminal or a nonterminal symbol. These are also sometimes called _tokens_.
  We note that a _token_ is also the name used for lexical units in ANTLR. We use both
  interchangeably when there is no ambiguity.

* A _rule_ is a finite sequence of _terms_ (two types of terms: terminals and
  nonterminals) that describe an expansion of a given terminal.

  For example, `[<term>+<expr>]` is one of the expansion rules of the nonterminal `<expr>`.

* A _definition_ is a set of _rules_ that describe the expansion of a given nonterminal.

  For example, `[[<digit>,<digits>],[<digit>]]` is the definition of the nonterminal `<digits>`

* A _context-free grammar_ is  composed of a set of nonterminals and 
  corresponding definitions that define the structure of the nonterminal.

  The grammar given below is an example context-free grammar.

* A terminal _derives_ a string if the string contains only the symbols in the
  terminal. A nonterminal derives a string if the corresponding definition
  derives the string. A definition derives the  string if one of the rules in
  the definition derives the string. A rule derives a string if the sequence
  of terms that make up the rule can derive the string, deriving one substring 
  after another contiguously (also called parsing).

* A *derivation tree* is an ordered tree that describes how an input string is
  derived by the given start symbol. Also called a *parse tree*.
  
* A derivation tree can be collapsed into its string equivalent. Such a string
  can be parsed again by the nonterminal at the root node of the derivation
  tree such that at least one of the resulting derivation trees would be the
  same as the one we started with.

We use the [fuzzingbook](https://www.fuzzingbook.org) *canonical* grammar style.

The format uses a JSON object (or equivalently a Python dictionary) for storing the grammar. The keys in the JSON object correspond to _nonterminals_ in the grammar. The object contains the _definition_ of each nonterminal. 

```python
grammar: {
    "<nonterminal>": definition,
    "<nonterminal>": definition
}
```

The _definition_ is stored as a list of _rules_.
```python
definition: [
    rule,
    rule
]
```
Each _rule_ is again a list of _terms_ which can be either terminal or nonterminal.
```python
rule: [terminal, nonterminal, terminal]
```

Each nonterminal symbol is a string that starts with `<` and ends with `>`, and cannot contain either of these characters inside.

```python
nonterminal: '<...>'
```

A terminal symbol is a string that contains any single character. A relaxation is that it can be any string so long as it does not start with `<` and ends with `>` and it is not empty.

A nonterminal that contains an empty definition in the grammar (an empty list) is considered to be undefined in the grammar.

Here is an example grammar for arithmetic expressions, starting at `<start>`.
A terminal symbol has exactly one character
(Note that we disallow empty string (`''`) as a terminal symbol).

In [ ]:
%%writefile src/grammars.py
import string

EXPR_GRAMMAR = {
 '<start>': [['<expr>']],
 '<expr>': [['<term>', '+', '<expr>'],
            ['<term>', '-', '<expr>'],
            ['<term>']],
 '<term>': [['<factor>', '*', '<term>'],
            ['<factor>', '/', '<term>'],
            ['<factor>']],
 '<factor>': [['+', '<factor>'],
              ['-', '<factor>'],
              ['(', '<expr>', ')'],
              ['<integer>', '.', '<integer>'],
              ['<integer>']],
 '<integer>': [['<digit>', '<integer>'], ['<digit>']],
 '<digit>': [[str(i)] for i in string.digits]}

EXPR_START = '<start>'

BEXPR_GRAMMAR = {
    '<start>': [['<bexpr>']],
    '<bexpr>': [
        ['<bop>', '(', '<bexprs>', ')'],
        ['<fault>']],
    '<bexprs>' : [['<bexpr>', ',', '<bexprs>'], ['<bexpr>']],
    '<bop>' : [list('and'), list('or'), list('neg')],
    '<fault>': [['<letters>'], []],
    '<letters>': [
        ['<letter>'],
        ['<letter>', '<letters>']],
    '<letter>': [[i] for i in (
        string.ascii_lowercase +
        string.ascii_uppercase +
        string.digits) + '_+*.-']
}
BEXPR_START = '<start>'

# Done

In [ ]:
#%tb